## Install necessary dependencies

In [ ]:
%pip install ultralytics opencv-python

## Import packages and configurations

In [ ]:
from ultralytics import YOLO
import glob
import cv2
import os

MODEL_PATH = "models/yolov12n-face.pt"
INPUT_DIR = "images/train"
OUTPUT_DIR = "images/annotated"
IMG_SIZE = 128
CONF_THRESHOLD = 0.25

os.makedirs(OUTPUT_DIR, exist_ok=True)

model = YOLO(MODEL_PATH)

## Run inference against training set

In [ ]:
for img_path in glob.glob(os.path.join(INPUT_DIR, "*.jpg")):
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️  Could not read {img_path}, skipping.")
        continue

    results = model(img, imgsz=IMG_SIZE, conf=CONF_THRESHOLD)
    res = results[0]

    annotated = img.copy()
    if res.boxes is not None and len(res.boxes) > 0:
        boxes = res.boxes.xyxy.cpu().numpy()
        confs = res.boxes.conf.cpu().numpy()

        for (x1, y1, x2, y2), conf in zip(boxes, confs):
            p1 = (int(x1), int(y1))
            p2 = (int(x2), int(y2))
            cv2.rectangle(annotated, p1, p2, (0, 255, 0), 2)
            cv2.putText(
                annotated,
                f"{conf:0.2f}",
                (p1[0], p1[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
                lineType=cv2.LINE_AA,
            )

    out_path = os.path.join(OUTPUT_DIR, os.path.basename(img_path))
    cv2.imwrite(out_path, annotated)
    num_faces = len(res.boxes) if res.boxes is not None else 0
    print(f"✅ {os.path.basename(img_path)} → {num_faces} face(s), saved to {out_path}")